In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier("F:\\Master OpenCV\\Haarcascades\\haarcascade_frontalface_default.xml")

def face_extractor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if faces is ():
        return None
    for (x, y ,w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face

cap = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count+=1
        face = cv2.resize(face_extractor(frame),(200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        file_path_name = "./face/user"+str(count)+'.jpg'
        cv2.imwrite(file_path_name,face)
        cv2.putText(face, str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
    else:
        print("No Face Found")
        pass

    if cv2.waitKey(0) & 0xFF == ord('q') or count == 100:
        break

In [ ]:
from os import listdir
from os.path import isfile, join

data_path = "./face/user"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]
Training_data, Labels = [], []
for i,files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.COLOR_BGR2GRAY)
    Training_data.append(np.asarray(images,np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels,np.int32)

model = cv2.face_LBPHFaceRecognizer()
model.train(np.asarray(Training_data),np.asarray(Labels))
print("model trained successfully")


In [ ]:
face_classifier = cv2.CascadeClassifier("F:\\Master OpenCV\\Haarcascades\\haarcascade_frontalface_default.xml")
def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h),(0,255,255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200,200))
        return img, roi

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = model.predict(face)
        if results[1] < 500:
            confidence = int(100 * (1 - (results[1]/300)))
            display_string = str(confidence) + '% Confident it is User'
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,120,150), 2)
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250,450), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,150), 2)
            cv2.imshow("Face Croper", image)
        else:
            cv2.putText(image, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,150), 2)
            cv2.imshow("Face Croper", image)
    except:
        cv2.putText(image, "No Face Fpund", (220,120), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,150), 2)
        cv2.putText(image, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,150), 2)
        cv2.imshow("Face Croper", image)
        pass
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

